# 1. 라이브러리 호출

In [105]:
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import time
import re
import csv 
import glob
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import seaborn as sns

from bs4 import BeautifulSoup
from tqdm import tqdm

In [106]:
#지역, 식당 URL을 저장하기위한 딕셔너리
place_url_dic={}
#지역+식당 장소만을 저장하기 위한 배열
place_name_list=[]
#최종 배열
place_url_arr=[]

# 2. 식당별 url 추출

In [107]:
def extract_url():
    #식당의 수를 세기 위한 변수(최대 75개)
    num=1
    #동적 크롤링을 위한 크롬드라이버
    driver = webdriver.Chrome(executable_path=r'D:\temp\chromedriver.exe')
    driver.implicitly_wait(6)

    #사용자로부터 원하는 지역 검색
    place = "배곧" 
    place = place.replace(' ', '') 
    place_url="https://m.map.naver.com/search2/search.naver?query={}맛집"

    #url접속
    driver.get(place_url.format(place))

    #식당 리스트를 가지고 있는 태그 추출

    #식당 이름 추출
    while num<=75:
        restaurant_name=driver.find_element_by_css_selector('#ct > div.search_listview._content._ctList > ul>li:nth-child({}) > div.item_info > a.a_item.a_item_distance._linkSiteview > div > strong'.format(num))
        restaurant_name_text=restaurant_name.text
        place_name_list.append(place+" "+restaurant_name_text)
        print(place+" "+restaurant_name_text)
        num+=1

    #개별 URL추출
    for i, keyword in enumerate(place_name_list):
        try:
            #검색 url 만들기
            naver_map_search_url = f'https://map.naver.com/v5/search/{keyword}/place' 
            driver.get(naver_map_search_url) #검색 url 접속, 즉 검색하기
            
            time.sleep(1) 
            cu = driver.current_url # 검색이 성공된 플레이스에 대한 개별 페이지
            
            res_code = re.findall(r"place/(\d+)", cu)
            final_url = 'https://pcmap.place.naver.com/restaurant/'+res_code[0]+'/review/visitor#'
            
            place_url_dic={'region':place, 'name':keyword,'url':final_url}
            place_url_arr.append(place_url_dic)
            print(place+" "+keyword+" : "+final_url)
        except IndexError:
            place_url_dic={'region':place, 'name':keyword,'url':'none'}
            place_url_arr.append(place_url_dic)
            print(place+" "+keyword+" : none")
    


## 2-(1) 식당별로 추출한 url csv 형태로 저장

In [108]:
def place_url_csv():
    with open('place_name_url.csv', 'w', newline='') as csv_file:
        fieldnames = ['region', 'name','url']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for data in place_url_arr:
            writer.writerow(data)

## 2-(2) 추출한 url의 csv파일 불러오기

In [109]:
#추출한 url의 csv파일 불러오기
place_url_list=pd.read_csv('place_name_url.csv',encoding='utf-8')
place_url=place_url_list[['region','name','url']]

In [110]:
place_url.shape

(75, 3)

## 2-(3) url이 none인 값 제거

In [111]:
#url이 none인 값 제거
data=pd.DataFrame(place_url)
data2=data[data.url!='none']

# 3. 네이버 플레이스 방문자 리뷰 정보 추출

In [112]:
visitor_review_dic={}

#방문자 리뷰 정보를 담기 위한 최종 배열
visitor_review_arr=[]

In [113]:
driver = webdriver.Chrome(executable_path=r'D:\temp\chromedriver.exe')
driver.implicitly_wait(4)

for idx,row in data2.iterrows():
    try:
        #식당 정보 url, 이름, 지역 가져오기
        url=row['url']
        place_name=row['name']
        region=row['region']
        print('========================')
        print(place_name + '식당')

        #url접속
        driver.get(url)
        driver.implicitly_wait(3)

        #더보기 버튼 다 누르기

        for i in range(3): #리뷰 100개씩만 가져오기 - 시간관계상
            try:
                driver.find_element_by_tag_name('body').send_keys(Keys.END) #맨 밑으로 내리기
                driver.find_element_by_css_selector('#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section._3fSeV > div._2kAri > a').click()
                driver.find_element_by_tag_name('body').send_keys(Keys.END)
                time.sleep(1)
            except NoSuchElementException:
                print("-더보기 버튼 모두 클릭 완료-")
                break

        #파싱
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        #print(soup)

        #식당 리뷰 개수 출력
        reviews = soup.find_all('li', attrs = {'class':'_3l2Wz'}) #-> _3l2Wz 이 부분은 네이버에서 주기적으로 바꿈.
        review_num = len(reviews) #특정 식당의 리뷰 총 개수
        print('리뷰 총 개수 : ' + str(review_num))

        #넘버링
        num = []
        for i in range(0, review_num):
            num.append(i+1)

        #크롤링 작업
        category = '방문자 리뷰'
        for i in range(0, review_num):
            # review 내용
            try : 
                review_content = reviews[i].find('span', attrs = {'class':'WoYOw'}).text

            except: #리뷰가 없다면
                review_content = ""

            visitor_review_dic = {
                'num':num[i],
                'name' : place_name,
                'blog or review' : category,
                'visitor review content' : review_content
            }
            visitor_review_arr.append(visitor_review_dic)
        
    except TimeoutException:
        print("TimeoutException")
        
    except NoSuchElementException:
        print("NoSuchElementException")

배곧 우정횟집식당
리뷰 총 개수 : 40
배곧 소쇄원 배곧본점식당


WebDriverException: Message: unknown error: cannot determine loading status
from unknown error: cannot determine loading status
from disconnected: received Inspector.detached event
  (Session info: chrome=101.0.4951.67)


## 3-(1) 네이버 플레이스 방문자 리뷰 정보 csv 형태로 저장

In [ ]:
def place_review_csv():
    with open('place_visitor_review.csv','w',encoding='utf-8-sig') as csv_file:
        fieldnames = ['num', 'name','blog or review','visitor review content']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for data in visitor_review_arr:
            writer.writerow(data)

# 3. 블로그 리뷰 정보 추출

In [117]:
import datetime
blog_review_dic={}

#블로그 리뷰 정보를 담기 위한 최종 배열
blog_review_arr=[]

error1 = []
driver = webdriver.Chrome(executable_path=r'D:\temp\chromedriver.exe')
driver.implicitly_wait(6)

for idx,row in data2.iterrows():
    try:
        url=row['url']
        place_name=row['name']
        print("========================")
        print(place_name)
        #블로그 리뷰를 추출하기 위해 url 변환
        # blog_url1 = url.replace('review/visitor#', 'home')
        # blog_url2 = blog_url1.replace('pcmap','m')
        # print(blog_url2)
        
        #변환한 url 접속
        blog_url = url.replace('review/visitor#', 'review/ugc')
        driver.get(blog_url)
        time.sleep(2)
        
        #식당 이름
        name_link = driver.find_element_by_xpath('//*[@id="_title"]/span[1]')
        name = name_link.text

        #음식점 링크에서 블로그 리뷰 버튼 클릭
    
        #블로그 리뷰 20개씩
        for i in range(1): 
            try:
                #driver.find_element_by_tag_name('body').send_keys(Keys.END) #맨 밑으로 내리기
                #time.sleep(1)
                #driver.find_element_by_css_selector('#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.no_margin._31amG > div._2kAri > a').click()
                #time.sleep(1)
                #driver.find_element_by_tag_name('body').send_keys(Keys.END)
                time.sleep(1)

            except NoSuchElementException:
                print('-더보기 버튼 모두 클릭 완료-')
                break


        #블로그 조회 페이지 파싱
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        time.sleep(1)

        #검색된 블로그 개수 조회
        count_review = soup.find_all('li', attrs = {'class':'awH2_'})
        blog_num=len(count_review)
        print('블로그 총 개수 : ' + str(blog_num))
        
        #각 블로그 주소 도출 및 저장
        url_list = []
        for i in range(0, len(count_review)): #각 블로그 주소 저장
            title_url = count_review[i].find('a', attrs = {'class':'_1cfgS'}).get('href')
            url_list.append(title_url)

        print(url_list)
        #각 블로그 내용 크롤링
        #content_list(String 값)에 모든 크롤링 리뷰 저장 후 real_content에 리스트화
        real_content = []
        blog_content = []
        blog_name = [] 
        like_count_list=[]
        command_count_list=[]
        date_list= []
        img=[]
        last_content_date_list = []
        neighber_count_list=[]
        for url in url_list: #수집한 url 만큼 반복
            #본문
            content_list = ""
            
            #블로그 이름
            name_list = ""
            
            #좋아요 수
            like_count=""

            #댓글 수
            command_count = ""

            #글 작성 날짜
            date=""

            #이미지 개수
            img_count=""
            
            #최신 게시물 날짜 또는 시간
            last_content_date=""
            
            #이웃 숫자
            neighber_count=""
            
            driver.get(url) #해당 url로 이동
            
            time.sleep(2)
            
            #본문
            overlays_content = ".se-component.se-text.se-l-default" #내용 크롤링
            contents = driver.find_elements_by_css_selector(overlays_content)

            #블로그 이름
            overlays_name = ".se-module.se-module-text.se-title-text"
            names = driver.find_elements_by_css_selector(overlays_name)

            #좋아요 수
            try:
                like_count=driver.find_element_by_css_selector('.btn_like').text
            except:
                like_count = str(0)
             
            #댓글 수
            try:
                command_count=driver.find_element_by_xpath('//*[@id="ct"]/div[4]/div[3]/div/div[2]/a[1]/em').text
            except:
                command_count=str(0)
                continue
              
            #글 작성 날짜
            date=driver.find_element_by_css_selector('.blog_date').text
            
            #이미지 개수
            try:
                img_count = len(driver.find_elements_by_css_selector('.se-module.se-module-image'))
            except:
                img_count = 0
                continue
            
            for content in contents:
                content_list = content_list + content.text #content_list 라는 값에 + 하면서 점점 누적
                content_list = content_list.replace("\n", '')
            for name in names:
                name_list = name.text
            
            try:
                driver.find_element_by_class_name('blog_thumbnail').click()
                time.sleep(0.5)
                print('블로그 메인으로 들어감')
                
                #card 아이콘 클릭
                time.sleep(0.5)
                driver.find_element_by_class_name('icon_card__OFkYm').click()   
                time.sleep(5)

                
                try:
                    last_content_date = driver.find_element_by_class_name('time__MHDWV').text
                    if(last_content_date[-1] == '전'):
                        last_content_date = datetime.datetime.now()
                        last_content_date = str(last_content_date.date())
                    else:
                        last_content_date = last_content_date.replace(". ", "-")
                        last_content_date = last_content_date.replace(".", "")
                except:
                    last_content_date = '해당 블로그의 글이 추가적으로 존재하지 않습니다.'
                
                #이웃 아이콘 클릭
                try:
                    time.sleep(0.5)
                    button =  driver.find_element_by_css_selector('.icon_buddy__h8xKE')
                    driver.execute_script("arguments[0].click();", button)
                    time.sleep(0.5)
                    
                    try:
                        neighber_count = driver.find_element_by_class_name('num').text
                    except:
                        neighber_count = '0'
                        print('num calss를 찾지 못했습니다')

                    
                except:
                    print('이웃 아이콘을 찾을 수 없습니다')
                    error = driver.current_url
                    error1.append(error)
                    print(driver.current_url)
                    neighber_count = '0'
                
                
            except:
                print('작가이름이 뭐죠.')
                last_content_date="블로그 작가 이름 못찾음 오류"
                neighber_count="블로그 작가 이름 못찾음 오류"
                
                
            

            print('blog_name: '+name_list)
            print('like: '+like_count)
            print('command_count_list: '+command_count)
            print('date: '+date)
            print('last_content_date: ' + last_content_date)
            print('neighber_count: ' + neighber_count)
            print('')
            
            blog_content.append(content_list)
            blog_name.append(name_list)
            like_count_list.append(like_count)
            command_count_list.append(command_count)
            date_list.append(date)
            img.append(img_count)
            last_content_date_list.append(last_content_date)
            neighber_count_list.append(neighber_count)
            time.sleep(1)

        # 블로그인지 리뷰인지 태그
        category = '블로그 리뷰'

        #넘버링
        num = []
        for i in range(0, len(blog_name)):
            num.append(i+1)

        for i in range(0, len(blog_name)):
            blog_review_dic = {
            'num':num[i],
            'name' : place_name,
            'blog or review' : category,
            'blog name' : blog_name[i],
            'like_count_list' : like_count_list[i],
            'command_count_list' : command_count_list[i],
            'date_list' : date_list[i],
            'img_count' : img[i],
            'neighber_count' : neighber_count_list[i],
            'last_content_date' : last_content_date_list[i],
            'blog review content' : blog_content[i]
            }
            blog_review_arr.append(blog_review_dic)
            print(blog_review_arr)
        
    except TimeoutException:
        print("TimeoutException")
        
    except NoSuchElementException:
        print("NoSuchElementException")
        

배곧 우정횟집
블로그 총 개수 : 10
['https://blog.naver.com/llcksdnjsll/222749154918', 'https://blog.naver.com/guswjd0407/222708267483', 'https://blog.naver.com/tkkmm2/222736209948', 'https://blog.naver.com/skiphard/222734697140', 'https://blog.naver.com/pww123/222714889405', 'https://blog.naver.com/hong3835/222275063088', 'https://blog.naver.com/wjdaks0522/222711380737', 'https://blog.naver.com/u_truth/221228949185', 'https://blog.naver.com/realjo25/222656037131', 'https://blog.naver.com/mnkj97/222656362832']
배곧 소쇄원 배곧본점
블로그 총 개수 : 10
['https://blog.naver.com/moamoasa/222686802472', 'https://blog.naver.com/dollo11/222325078690', 'https://blog.naver.com/lightsout/222283500271', 'https://blog.naver.com/urban_girls/222535518496', 'https://blog.naver.com/lotto0513/222354246110', 'https://blog.naver.com/thwjd6725/222232769898', 'https://blog.naver.com/evoss_aircell/222710886848', 'https://blog.naver.com/nalpamzo/222741265223', 'https://blog.naver.com/bromom_sonncyann/222715843277', 'https://blog.naver.

WebDriverException: Message: chrome not reachable
  (Session info: chrome=101.0.4951.67)


In [ ]:
with open('place_blog_review.csv','w',encoding='utf-8-sig') as csv_file:
    fieldnames = ['num', 'name','blog or review','blog name','like_count_list','command_count_list','date_list','img_count','neighber_count','last_content_date','blog review content']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for data in blog_review_arr:
        writer.writerow(data)